# 2. RAG Question Answering for Car Intelligent Assistant

Having a vector dataset generated with step 1. This code shows how to perform questions and get answers augmented with context from the dataset. This is called "Retrieval Augmented Generation" or RAG.

Authors:
- Luis Bernardo Hernandez Salinas
- Juan R. Terven

In [7]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from getpass import getpass
from langchain_core.prompts import ChatPromptTemplate

## Set API, embedding and vector database path

In [24]:
# List of models tested:
# gpt-3.5-turbo
# gpt-4-turbo
# gpt-4o
# claude-3-haiku-20240307
# claude-3-sonnet-20240229
# claude-3-opus-20240229
# open-mistral-7b
# open-mixtral-8x7b
# open-mixtral-8x22b

# Model to use
llm_name = "open-mixtral-8x22b"

embedding_dimensions = 1536 #1536  # 3072

# API key 
if "gpt" in llm_name:
    client = os.environ['OPENAI_API_KEY']
elif "claude" in llm_name:
    client = os.environ['ANTHROPIC_API_KEY']
elif "mistral" in llm_name or "mixtral" in llm_name:
    client = os.environ['MISTRAL_API_KEY']
else:
    print("INVALID MODEL!")
    
print(f"Using model {llm_name}")

embedding_model = OpenAIEmbeddings()

# Vector dataset
vectordb_directory = f'vector_database_chspark_{embedding_dimensions}'
print(f"Using vector database {vectordb_directory}")

Using model open-mixtral-8x22b
Using vector database vector_database_chspark_1536


## Load Vector database

In [25]:
# Create chroma db from existing vectordb_directory
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory=vectordb_directory
)

print(f"Load {vectordb._collection.count()} collections from vector database")


Load 243 collections from vector database


## Model instruction for Retrieval Augmented Generation

In [26]:
template = """\
You are an assistant and you have to answer the questions that are asked of you. 
If the question is about the vehicle, use the provided car manual information to answer the question at the end. 
If you don’t know the answer even with the car manual provided say "I am sorry, I did not find the answer in the car manual"
Don’t try to make up an answer.
Respond in the most attentive way possible. Use a maximum of three sentences. 
Keep the answer as concise as possible. 
Always say “Do you have any other questions?” at the end of the answer!
Context: {context}
Question: {question}
Helpful Answer:
"""

### Set LLM and RAG object

In [27]:
# create prompt template object
qa_chain_prompt = PromptTemplate.from_template(template)

if "gpt" in llm_name:
    llm = ChatOpenAI(model_name=llm_name, temperature=0) 
elif "claude" in llm_name:    
    llm = ChatAnthropic(model_name=llm_name, api_key=client, temperature=0)
elif "mistral" in llm_name or "mixtral" in llm_name:
    llm = ChatMistralAI(model=llm_name, api_key=client, temperature=0)

print(llm)

# QA RAG object
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)

client=<httpx.Client object at 0x000001BE27019ED0> async_client=<httpx.AsyncClient object at 0x000001BE2846BA50> mistral_api_key=SecretStr('**********') model='open-mixtral-8x22b' temperature=0.0


## Let's try RAG with a query

In [28]:
#query = "what is the recommended fuel for this vehicle?"
query = "should a pregnant woman use the seatbelts?"

# Run RAG chain: 
# 1. Take the query and add it to the prompt
# 2. Get query embedding
# 3. Retrieve the most relevant documents based on embedding similarity
# 4. Augment the prompt with the retrieved documents
# 5. Send prompt to LLM
# 6. Get model response
model_response = qa_chain.invoke({"query": query})

print(model_response["result"])


Yes, a pregnant woman should use the seatbelts. The lap belt should be positioned below the belly, across the hips, and the shoulder belt should be placed across the chest, between the breasts, and away from the neck. This is to ensure safety during travel. Do you have any other questions?
